In [1]:
def omega_func(eta, x_bar, u_bar, e=1):
    return x_bar**(eta - 1)*s*(1 - u_bar)/(u_bar*e)

In [2]:
def rho_func(eta, s, u, x, omega, tau):
    return omega*x**(-1*eta)*tau/((1 + tau)*s)

In [3]:
def tau_bar_func(eta, u_bar):
    return (1 - eta)*u_bar/eta

In [4]:
def q_func(x, **params):
    omega, eta = params['omega'], params['eta']
    return omega*x**(-eta)

In [5]:
def tau_func(x, **params):
    s, rho = params['s'], params['rho']
    return s*rho/(q_func(x=x, **params) - s*rho)

In [6]:
def u_func(x, **params):
    s, eta, omega = params['s'], params['eta'], params['omega']
    f = omega*x**(1 - eta)
    return s/(s + f)

In [7]:
def Y_func(x, **params):
    s, eta, omega, k = params['s'], params['eta'], params['omega'], params['k']
    return (1 - u_func(x=x, **params))*k

In [8]:
def dlnydlnx_func(x, **params):
    eta = params['eta']
    return (1 - eta)*u_func(x=x, **params) - eta*tau_func(x=x, **params)

In [9]:
GY_func = lambda GC:GC/(1 + GC) # G/Y
CY_func = lambda GC:1 - GY_func(GC)  # C/Y
GC_func = lambda GY:GY/(1 - GY) # G/C

In [10]:
def U_func(c, g, **params):
    epsilon, gamma = params['epsilon'], params['gamma']
    if epsilon == 1:
        scalar = (1 - gamma)**(1 - gamma)*gamma**gamma
        return c**(1 - gamma)*g**(gamma)/scalar
    else:
        return ((1 - gamma)**(1/epsilon)*c**((epsilon - 1)/epsilon) + 
                gamma**(1/epsilon)*g**((epsilon - 1)/epsilon))**(epsilon/(epsilon - 1))

In [11]:
def dUdc_func(gc, **params):
    epsilon, gamma = params['epsilon'], params['gamma']
    return ((1-gamma)*U_func(c=1, g=gc, **params))**(1/epsilon)
def dlnUdlnc_func(gc, **params):
    epsilon, gamma = params['epsilon'], params['gamma']
    return (1 - gamma)**(1/epsilon)*(U_func(c=1, g=gc, **params))**((1 - epsilon)/epsilon)

In [12]:
def dlnUdlng_func(gc, **params):
    epsilon, gamma = params['epsilon'], params['gamma']
    return gamma**(1/epsilon)*(U_func(1/gc, 1, **params))**((1 - epsilon)/epsilon)

In [13]:
def MRS_func(gc, **params):
    epsilon, gamma = params['epsilon'], params['gamma']
    return gamma**(1/epsilon)/(1-gamma)**(1/epsilon)*gc**(-1/epsilon)

In [14]:
def dlnUcdlnc_func(gc, **params):
    epsilon = params['epsilon']
    return (dlnUdlnc_func(gc, **params) - 1)/epsilon
def dlnUcdlng_func(gc, **params):
    epsilon = params['epsilon']
    return dlnUdlng_func(gc, **params)/epsilon

In [15]:
def p_func(G, **params):
    r, p0, Y_bar = params['r'], params['p0'], params['Y_bar']
    return p0*dUdc_func(gc=G/(Y_bar-G), **params)**(1 - r)

In [16]:
def dlnpdlng_func(G, **params):
    r,  Y_bar = params['r'],  params['Y_bar']
    return (1 - r)*(dlnUcdlng_func(gc=G/(Y_bar - G), **params) - dlnUcdlnc_func(gc=G/(Y_bar - G), **params)*(G/(Y_bar - G)))

In [17]:
def dlnxdlng_func(G, x, **params):
    eta = params['eta']
    tau = tau_func(x=x, **params)
    Y = Y_func(x=x, **params)
    dlncdlnx = eta*tau/dlnUcdlnc_func(gc=G/(Y-G), **params)
    dlncdlng = (dlnpdlng_func(G=G, **params)-dlnUcdlng_func(gc=G/(Y - G), **params))/dlnUcdlnc_func(gc=G/(Y - G), **params)
    return (G/Y + (1 - G/Y)*dlncdlng)/(dlnydlnx_func(x=x, **params) - (1-G/Y)*dlncdlnx)

In [18]:
def m_func(which, **params):
    if which == 'M':
        u, M, GY, eta, tau = params['u'], params['M'], params['GY'], params['eta'], params['tau']
        return (1 - u)*M/(1 - GY*eta/(1 - eta)*tau/u*M)
    elif which == 'dlnxdlng':
        G, x, eta = params['G'], params['x'], params['eta']
        q = q_func(**params)
        u = u_func(**params)
        tau = tau_func(**params)
        Y = Y_func(**params)
        dlnxdlng = dlnxdlng_func(**params)
        return (1 - eta)*u*(1 - u)*dlnxdlng*Y/G 

In [19]:
def M_func(G, x, **params):
    eta = params['eta']
    q = q_func(x=x, **params)
    u = u_func(x=x, **params)
    tau = tau_func(x=x, **params)
    m = m_func(which='dlnxdlng', G=G, x=x, **params)
    return m/(1 - u + G/Y_func(x, **params)*eta*tau/(1 - eta)/u*m)

In [20]:
def find_eq(G, x, alpha, **params):
    s, rho, omega, eta = params['s'], params['rho'], params['omega'], params['eta']
    return abs(dUdc_func(G/(Y_func(x, **params) - G), **params) - ((1 + tau_func(x=x, **params))*p_func(G, **params)/alpha))

In [21]:
def optimal_func(G, x, **params):
    return abs(1 - MRS_func(gc=G/(Y_func(x, **params) - G), **params) - dlnydlnx_func(x=x, **params)*dlnxdlng_func(G=G, x=x, **params)*Y_func(x=x, **params)/G)

In [22]:
def suffstat_func(u0, m, **params):
    u_bar, z0, z1, epsilon = params['u_bar'], params['z0'], params['z1'], params['epsilon']
    du = (u0 - u_bar)/u_bar
    return epsilon*z0*m/(1 + z1*z0*epsilon*m**2)*du